In [2]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import time

nft_collection_df = pd.read_csv('eth_nft_30days.csv') #get sigdev nft rankings for the 30days

In [220]:
# cleaning the csv. file to the right format
for index, row in nft_collection_df.iterrows():
    mystring =row['COLLECTION'].split(">",1)[1]
    mystring = mystring.split("<")[0]
    mystring = mystring.replace("'", " ")
    mystring = mystring.replace("[", " ")
    mystring = mystring.replace("]", " ")
    mystring = mystring.replace(":", " ")
    mystring = mystring.replace("  ", " ")
    mystring = mystring.replace(" - ", " ")

    nft_collection_df.at[index, 'collection'] = mystring

In [222]:
# filtering out some nft collections
nft_collection_df['collection'] = np.where((nft_collection_df.collection == 'Sandbox s LANDs'),'Sandbox Land',nft_collection_df.collection)
nft_collection_df['collection'] = np.where((nft_collection_df.collection == 'Primera by Mitchell and Yun'),'Primera',nft_collection_df.collection)

#not enough data for 0'clocks nft
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == '7 O clock'].index, inplace = True)
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == '3 O clock'].index, inplace = True)
#sql error -  bind parameter
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'Dci'].index, inplace = True) 
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'AK Ultra'].index, inplace = True) 
# twitter api data = 0 error {'meta': {'result_count': 0}}
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'VOX Series 3'].index, inplace = True)
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'Beeple Spring Collection'].index, inplace = True)
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'Bored & Dangerous'].index, inplace = True)

In [10]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import time
import os

# step 1: connect to Postgres Socials Data/ Twitter API
engine = db.create_engine("")
connection = engine.connect()
bearer_token = '' #twitter bearer token

# step 2: get distinct collection names to track (split them up)
nft_dict = dict(zip(nft_collection_df.collection, nft_collection_df.index))
top_100_collections = list(nft_dict.keys())
nested_top_100_collections = [top_100_collections[n:n+10] for n in range(0, len(top_100_collections), 10)] #splitting them up for twitter rate-limit

# Twitter Query
for i in range(len(nested_top_100_collections)): # paginating to account for twitter api rate limit 
    for item in nested_top_100_collections[i]:
        # print(item)
        headers = {'Authorization': f"Bearer {bearer_token}",}
        params = {
            'query': item + ' "nft" -is:retweet',
            "tweet.fields": "created_at,public_metrics,text",
            "sort_order": "relevancy",
            "max_results": 100,
            "expansions": "author_id",
        }
        response = requests.get('https://api.twitter.com/2/tweets/search/recent', params=params, headers=headers)
        data = response.json()
        data = pd.DataFrame(data['data'])
        data = pd.concat([data.drop(['public_metrics'], axis=1), data['public_metrics'].apply(pd.Series)], axis=1)
        result = data[['created_at', 'author_id', 'text','retweet_count', 'reply_count', 'like_count', 'id']]
        result = result.replace('\n',' ', regex=True)
        result = result.replace("'"," ", regex=True)
        result = result.replace("'s", " s", regex=True)
        
        # making it into a string of tuple to insert sql command
        s = ', '.join(["("+ ', '.join( ["'" + item + "'", "'" + str(result.iloc[i].created_at)[0:19]+ "'",
              "'"+str(result.iloc[i].author_id)+"'", "'"+str(result.iloc[i].text)+"'", "'" + str(result.iloc[i].retweet_count)+ "'", "'"+str(result.iloc[i].reply_count)+"'", "'"+str(result.iloc[i].like_count)+"'", "'"+str(result.iloc[i].id)+"'"+ ")" ]) for i in range(len(result))])
        
        query = f"""
        INSERT INTO nft_tweets_demo 
        VALUES {s}
        ON CONFLICT ON CONSTRAINT unique_tweet_id 
        DO NOTHING
        """
        try:
            pd.read_sql_query(db.text(query), connection)

        except Exception as e:
            print(e)
            print(item)

    # print('---loop---')

LILVERSE
Invisible Friends
WINGS GENESIS
ALPACADABRAZ
Something
CloneX
Yakuza Pandas
Great Expectations
PudgyPenguins
MultibeastsByHaas
8PES
goblintown
NetvrkLand
SHIBOSHIS
Azimuth Points
Jenkins the Valet
Art Blocks
Impostors Genesis
Cool Cats
GoblinVerse
STOICS
omgkirby x CT
Spooktaculars
Cold Blooded Creepz
SAN Origin
ENS
DeadFellaz
Hashmasks
parallel
Emblem Vault V2
Neo Tokyo Citizen
Unidentified contract
Elves Forest NFT
BoredApeYachtClub
Lazy Ape Yacht Club
Frank Stella Geometries
World Of Women
Akutars
Dooplicator
FewoWorld Paint
Gala Games
CryptoDickbutts S3
HsGenesisPass
The Plague
y00ts Yacht Club
Welly Friends
mfer
The Art of Seasons
Looki
Primera
Azuki
Sandbox Land
Imaginary Ones
CyberKongz VX
Cybernetic Harambes
DigiDaigakuSpirits
Gutter Dogs
CNP Jobs
DEGEN TOONZ
VeeFriends Series 2
Porkers
Judy Rabbit
VaynerSports Pass
ATARI50
Wolf Game
Moonbirds Oddities
Keungz Genesis
MetaHero
Hume Genesis
golden egg club
Otherdeed
posers
YOLO Bunny
10KTF
Metamorphic.SevenTreasures
Pota